In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.2 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.7.0


In [ ]:
fash_mnst = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels),(test_images,test_labels) = fash_mnst.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
# scale down the images from 0-1 as it is grey scale images
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
# converting or reshaping
train_images =train_images.reshape(len(train_images),28,28,1)
test_images =test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model, objective='val_accuracy',max_trials=5,directory='output', project_name = 'Fashion MNSIT')

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 06m 23s]
val_accuracy: 0.8831666707992554

Best val_accuracy So Far: 0.8831666707992554
Total elapsed time: 00h 07m 16s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |112               |32                
conv_1_kernel     |3                 |3                 
conv_2_filter     |48                |48                
conv_2_kernel     |5                 |3                 
dense_1_units     |112               |80                
learning_rate     |0.01              |0.01              

Epoch 1/3
1688/1688 [==============================] - 618s 366ms/step - loss: 0.5331 - accuracy: 0.8247 - val_loss: 0.3792 - val_accuracy: 0.8572
Epoch 2/3
1688/1688 [==============================] - 617s 366ms/step - loss: 0.3640 - accuracy: 0.8650 - val_loss: 0.3743 - val_accuracy: 0.8658
Epoch 3/3
 506/1688 [=======>......................] - ETA: 7:02 - loss: 0.3223 - accuracy: 0.8775

In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        48432     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 128)               2973824   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 3,026,458
Trainable params: 3,026,458
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1,initial_epoch=3 )

Epoch 4/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.1595 - accuracy: 0.9403 - val_loss: 0.2580 - val_accuracy: 0.9125
Epoch 5/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1214 - accuracy: 0.9548 - val_loss: 0.2764 - val_accuracy: 0.9155
Epoch 6/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0914 - accuracy: 0.9654 - val_loss: 0.3114 - val_accuracy: 0.9150
Epoch 7/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0666 - accuracy: 0.9755 - val_loss: 0.3868 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0514 - accuracy: 0.9814 - val_loss: 0.4134 - val_accuracy: 0.9142
Epoch 9/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0397 - accuracy: 0.9856 - val_loss: 0.5030 - val_accuracy: 0.9113
Epoch 10/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0347 - accuracy: 0.9878 - val_loss: 0.5112 